Regression Models with Keras

In [2]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
%pip install tensorflow_cpu==2.18.0


   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   - -------------------------------------- 11.5/390.3 MB 60.3 MB/s eta 0:00:07
   --- ------------------------------------ 32.0/390.3 MB 81.3 MB/s eta 0:00:05
   ----- ---------------------------------- 55.6/390.3 MB 95.7 MB/s eta 0:00:04
   -------- ------------------------------ 80.7/390.3 MB 101.0 MB/s eta 0:00:04
   ---------- --------------------------- 104.3/390.3 MB 104.1 MB/s eta 0:00:03
   ------------ ------------------------- 129.5/390.3 MB 106.0 MB/s eta 0:00:03
   -------------- ----------------------- 149.9/390.3 MB 106.5 MB/s eta 0:00:03
   ---------------- --------------------- 167.0/390.3 MB 101.7 MB/s eta 0:00:03
   ------------------ ------------------- 192.2/390.3 MB 104.1 MB/s eta 0:00:02
   -------------------- ----------------- 215.5/390.3 MB 105.1 MB/s eta 0:00:02
   ----------------------- -------------- 240.6/390.3 MB 106.1 MB/s eta 0:00:02
   ------------------------- ------------ 265.6/3

  You can safely remove it manually.
  You can safely remove it manually.


In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np

import keras

In [6]:
#Download the dataset
filepath='https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv'
concrete_data = pd.read_csv(filepath)

concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [8]:
concrete_data.shape

(1030, 9)

In [11]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [25]:
#Split into features and labels
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']]
target = concrete_data['Strength']
predictors.head()
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [26]:
#Normalize the data
#!pip install scikit-learn
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(predictors)  # Compute the mean and std for scaling
predictors_normalized = scaler.transform(predictors)  # Apply the normalization

# Convert back to DataFrame
predictors_normalized = pd.DataFrame(predictors_normalized, columns=predictors.columns)

# Display the normalized data
print(predictors_normalized.head())    

     Cement  Blast Furnace Slag   Fly Ash     Water  Superplasticizer  \
0  2.477915           -0.856888 -0.847144 -0.916764         -0.620448   
1  2.477915           -0.856888 -0.847144 -0.916764         -0.620448   
2  0.491425            0.795526 -0.847144  2.175461         -1.039143   
3  0.491425            0.795526 -0.847144  2.175461         -1.039143   
4 -0.790459            0.678408 -0.847144  0.488793         -1.039143   

   Coarse Aggregate  Fine Aggregate       Age  
0          0.863154       -1.217670 -0.279733  
1          1.056164       -1.217670 -0.279733  
2         -0.526517       -2.240917  3.553066  
3         -0.526517       -2.240917  5.057677  
4          0.070527        0.647884  4.978487  


In [27]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

In [30]:
#Build the model function
def regression_model():
    model = Sequential()
    model.add(Input(shape=(predictors_normalized.shape[1],)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    
    return model


In [29]:
#Train and Test the NN
model = regression_model()
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 50)             │           450 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50)             │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,051 (11.92 KB)

 Trainable params: 3,051 (11.92 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.fit(predictors_normalized, target, epochs=100, verbose=2, validation_split=0.3)

Epoch 1/100
23/23 - 1s - 22ms/step - loss: 1675.7332 - mae: 36.8661 - val_loss: 1169.5677 - val_mae: 31.4718
Epoch 2/100
23/23 - 0s - 3ms/step - loss: 1553.7095 - mae: 35.2745 - val_loss: 1055.2920 - val_mae: 29.7048
Epoch 3/100
23/23 - 0s - 3ms/step - loss: 1353.0939 - mae: 32.5048 - val_loss: 868.5073 - val_mae: 26.5746
Epoch 4/100
23/23 - 0s - 3ms/step - loss: 1050.2079 - mae: 27.9140 - val_loss: 626.0470 - val_mae: 21.9132
Epoch 5/100
23/23 - 0s - 3ms/step - loss: 691.4441 - mae: 21.5337 - val_loss: 390.5170 - val_mae: 16.3943
Epoch 6/100
23/23 - 0s - 3ms/step - loss: 410.4359 - mae: 16.0455 - val_loss: 243.8016 - val_mae: 12.6711
Epoch 7/100
23/23 - 0s - 3ms/step - loss: 278.9724 - mae: 13.3812 - val_loss: 202.4815 - val_mae: 11.5990
Epoch 8/100
23/23 - 0s - 3ms/step - loss: 245.0592 - mae: 12.6209 - val_loss: 196.0741 - val_mae: 11.4153
Epoch 9/100
23/23 - 0s - 3ms/step - loss: 230.1090 - mae: 12.2525 - val_loss: 193.8599 - val_mae: 11.3353
Epoch 10/100
23/23 - 0s - 3ms/step - lo